In [ ]:
# Install necessary libraries
!pip install transformers datasets
!pip install torch
!pip install --upgrade huggingface_hub


In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('colab')
login(token)



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load model directly
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", use_auth_token=token)
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", use_auth_token=token)

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf")

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

def generate_story(prompt, max_length=200):
    output = generator(prompt, max_length=max_length, num_return_sequences=1, truncation=True)
    return output[0]['generated_text'] if isinstance(output, list) else output

def continue_story(existing_text, max_new_tokens=100):
    continuation = generator(existing_text, max_new_tokens=max_new_tokens, num_return_sequences=1, truncation=True)
    return continuation[0]['generated_text'] if isinstance(continuation, list) else continuation

def generate_alternatives(existing_text, max_new_tokens=100, num_alternatives=3):
    alternatives = generator(existing_text, max_new_tokens=max_new_tokens, num_return_sequences=num_alternatives, truncation=True)
    return [alt['generated_text'] for alt in alternatives]

def erase_and_rewrite(new_content, max_length=200):
    return generate_story(new_content, max_length=max_length)

def generate_story_with_style(prompt, style, max_length=200):
    # Adjust the prompt or settings based on the style
    if style == "fantasy":
        prompt = f"In a magical land, {prompt}"
    elif style == "sci-fi":
        prompt = f"In a distant future, {prompt}"
    # Add other styles similarly
    return generator(prompt, max_length=max_length, num_return_sequences=1)[0]['generated_text']

def suggest_corrections(story_text):
    # This is a simple example where you identify sentences that could be improved
    sentences = story_text.split('.')
    suggestions = [f"Consider revising this sentence: '{s.strip()}.'" for s in sentences if len(s) > 100]
    return suggestions

def interactive_story_writing():
    print("Welcome to the Story Writing Assistant!")
    story = ""

    while True:
        user_input = input("Enter your prompt or command (type 'style:fantasy' or 'style:sci-fi' , 'continue' , 'alternatives', 'erase and rewrite', 'suggest correction', or  'end' to finish): ")

        if user_input.lower() == "end":
            print("Final Story:")
            print(story)
            break

        elif user_input.lower().startswith("style:"):
            # Command to generate a story with a specific style
            parts = user_input.split(":", 2)
            if len(parts) < 3:
                print("Please provide both a style and a prompt. For example: 'style: fantasy: In a magical land,'")
                continue

            style, prompt = parts[1].strip(), parts[2].strip()
            story = generate_story_with_style(prompt, style)
            print("Story with Style:")
            print(story)

        elif user_input.lower() == "continue":
            # Command to continue the current story
            continuation = continue_story(story)
            story += continuation
            print("Continued Story:")
            print(story)

        elif user_input.lower() == "alternatives":
            # Command to generate alternative continuations
            alternatives = generate_alternatives(story)
            for i, alt in enumerate(alternatives, 1):
                print(f"Alternative {i}:\n{alt}\n")
            choice = int(input(f"Choose an alternative (1-{len(alternatives)}) or 0 to keep the current story: "))
            if choice > 0 and choice <= len(alternatives):
                story = alternatives[choice - 1]
            print("Updated Story:")
            print(story)

        elif user_input.lower().startswith("erase and rewrite:"):
            # Command to erase and rewrite the entire story
            if ":" in user_input:
                _, new_content = user_input.split(":", 1)
                story = erase_and_rewrite(new_content.strip())
                print("Updated Story:")
                print(story)
            else:
                print("Error: Please provide new content after 'erase and rewrite:'.")


        elif user_input.lower() == "suggest corrections":
            # Command to suggest corrections
            suggestions = suggest_corrections(story)
            for suggestion in suggestions:
                print(suggestion)

        else:
            # Default behavior: add new text to the story
            if story == "":
                story = generate_story(user_input)
            else:
                story += " " + generate_story(user_input)
            print("Current Story:")
            print(story)

# Run the interactive session
interactive_story_writing()



tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]